Realizando importações para o funcionamento da aplicação

In [7]:
import pyspark
import os
import pyspark
from delta import *
from delta.tables import *
from pyspark.sql.functions import *

Criando uma sessão do Spark com Delta Lake

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("DeltaLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 'org.apache.hadoop:hadoop-aws:3.3.4') \
    .config('spark.hadoop.fs.s3a.endpoint', "http://minio:9000") \
    .config('spark.hadoop.fs.s3a.access.key', "admin") \
    .config('spark.hadoop.fs.s3a.secret.key', "password") \
    .config('spark.hadoop.fs.s3a.path.style.access', "true") 


In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eae98d78-c799-4e9c-a130-af2e21a09d27;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (6679ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (364ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (449ms)
:: resolution report :: resolve 5977ms :: artifacts dl

Importando base de dados e criando tabela no formato Delta no Min.IO

In [4]:
print("Iniciando criação da tabela...")

root = os.path.dirname(os.getcwd())

climate_change_df = spark.read.parquet("{0}/data/climate_change.parquet".format(root))
climate_change_df.write.save(mode="overwrite", path="s3a://warehouse/deltalake_tables", format="delta")

Iniciando criação da tabela...


24/04/25 20:13:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/04/25 20:13:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/04/25 20:14:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Checando os dados da tabela recém criada dentro do Min.IO

In [5]:
cc_df = spark.read.load("s3a://warehouse/deltalake_tables", format="delta")

cc_df.filter((cc_df["City"] == "Porto Alegre") & (cc_df["Country"] == "Brazil")).show(5)

+----------+------------------+-----------------------------+------------+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|        City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+------------+-------+--------+---------+
|1832-01-01|            23.173|                        1.692|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-02-01|             22.93|                        2.661|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-03-01|            21.281|                        1.826|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-04-01|            17.697|                        2.482|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-05-01|            14.214|                        2.263|Porto Alegre| Brazil|  29.74S|   51.69W|
+----------+------------------+-----------------------------+------------+-------+--------+---------+
only showing top 5 rows



Atualizando dados na tabela

In [8]:
cc_table = DeltaTable.forPath(spark, "s3a://warehouse/deltalake_tables")

cc_table.update(
    condition=expr("City == 'Porto Alegre'"),
    set={"Country": lit("Exemplo Update")})

cc_df = cc_table.toDF()
cc_df.filter(cc_df["City"] == "Porto Alegre").show(5)

+----------+------------------+-----------------------------+------------+--------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|        City|       Country|Latitude|Longitude|
+----------+------------------+-----------------------------+------------+--------------+--------+---------+
|1832-01-01|            23.173|                        1.692|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-02-01|             22.93|                        2.661|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-03-01|            21.281|                        1.826|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-04-01|            17.697|                        2.482|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-05-01|            14.214|                        2.263|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
+----------+------------------+-----------------------------+------------+--------------+--------+---------+
only showing top 5 

Deletando dados da tabela

In [9]:
cc_table = DeltaTable.forPath(spark, "s3a://warehouse/deltalake_tables")

cc_table.delete(expr("City == 'Porto Alegre'"))

cc_df = cc_table.toDF()
cc_df.filter(cc_df["City"] == "Porto Alegre").show(5)

+---+------------------+-----------------------------+----+-------+--------+---------+
| dt|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|
+---+------------------+-----------------------------+----+-------+--------+---------+
+---+------------------+-----------------------------+----+-------+--------+---------+



Demonstração da funcionalidade de histórico do Delta Lake

Tabela anterior a atualização

In [10]:
df_zero = spark.read.format("delta").option("versionAsOf", 0).load("s3a://warehouse/deltalake_tables")
df_zero.filter(df_zero["City"] == "Porto Alegre").show(5)

+----------+------------------+-----------------------------+------------+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|        City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+------------+-------+--------+---------+
|1832-01-01|            23.173|                        1.692|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-02-01|             22.93|                        2.661|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-03-01|            21.281|                        1.826|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-04-01|            17.697|                        2.482|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-05-01|            14.214|                        2.263|Porto Alegre| Brazil|  29.74S|   51.69W|
+----------+------------------+-----------------------------+------------+-------+--------+---------+
only showing top 5 rows



Tabela após a atualização e anterior ao delete

In [11]:
df_um = spark.read.format("delta").option("versionAsOf", 1).load("s3a://warehouse/deltalake_tables")
df_um.filter(df_um["City"] == "Porto Alegre").show(5)

+----------+------------------+-----------------------------+------------+--------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|        City|       Country|Latitude|Longitude|
+----------+------------------+-----------------------------+------------+--------------+--------+---------+
|1832-01-01|            23.173|                        1.692|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-02-01|             22.93|                        2.661|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-03-01|            21.281|                        1.826|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-04-01|            17.697|                        2.482|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-05-01|            14.214|                        2.263|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
+----------+------------------+-----------------------------+------------+--------------+--------+---------+
only showing top 5 

Tabela após o delete

In [12]:
df_dois = spark.read.format("delta").option("versionAsOf", 2).load("s3a://warehouse/deltalake_tables")
df_dois.filter(df_dois["City"] == "Porto Alegre").show(5)

+---+------------------+-----------------------------+----+-------+--------+---------+
| dt|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|
+---+------------------+-----------------------------+----+-------+--------+---------+
+---+------------------+-----------------------------+----+-------+--------+---------+

